In [1]:
#%pip install xrpl-py

In [13]:
#%pip install asyncio

In [4]:
#Establish connection to testnet
from xrpl.asyncio.clients import AsyncJsonRpcClient

client = AsyncJsonRpcClient(url="https://s.altnet.rippletest.net:51234/")
print(client)

In [16]:
#Create issuer and receiver wallets and prefund them 
from xrpl.asyncio.wallet import generate_faucet_wallet

issuer = await generate_faucet_wallet(client)
receiver = await generate_faucet_wallet(client)
print(issuer, receiver)


C:\Users\Kris\anaconda3\envs\xrpl_env\lib\typing.py:1119: RuntimeWarning: coroutine 'generate_faucet_wallet' was never awaited
  params = tuple(_type_check(p, msg) for p in params)
C:\Users\Kris\AppData\Local\Temp/ipykernel_10776/543330420.py:3: RuntimeWarning: coroutine 'generate_faucet_wallet' was never awaited
  issuer = await generate_faucet_wallet(client)


public_key: EDD0782CAE3F042E194D94377B3FAD7BC4CE9D850BD4E5C352861FB18BBFEB41EA
private_key: -HIDDEN-
classic_address: r9WZ5PdJHyL1LodqQxL4TsKnRSW2uEGUNi public_key: ED88C40AFB411C8A51F1F57A555D83D27A57FCDB180E88FD2657D2CC51FC7DBF5B
private_key: -HIDDEN-
classic_address: rwHtUmDVoFjP2jBCQcuWuRDbVU26svAGk9


C:\Users\Kris\AppData\Local\Temp/ipykernel_10776/543330420.py:4: RuntimeWarning: coroutine 'generate_faucet_wallet' was never awaited
  receiver = await generate_faucet_wallet(client)


In [7]:
#Construct token_symbol by converting name into hex (needed for >3 letter name)
token = "ZINC"
token_bytes = token.encode("ascii").hex().upper()
token_symbol = '{:<040s}'.format(token_bytes)
amount = "13000"
print(token_symbol)

5A494E4300000000000000000000000000000000


In [19]:
#Checking latest closed ledger
from xrpl.asyncio.ledger import get_latest_validated_ledger_sequence
print(await get_latest_validated_ledger_sequence(client))

24181232


In [23]:
from xrpl.models.transactions import AccountSet, AccountSetFlag
from xrpl.asyncio.transaction import safe_sign_and_autofill_transaction, send_reliable_submission
#Set default ripple flag?
account_set = AccountSet(account=issuer.classic_address, set_flag= AccountSetFlag.ASF_DEFAULT_RIPPLE)
signed_account_set = await safe_sign_and_autofill_transaction(account_set, issuer, client)
response = await send_reliable_submission(signed_account_set, client)
print(response)

Response(status=<ResponseStatus.SUCCESS: 'success'>, result={'Account': 'r9WZ5PdJHyL1LodqQxL4TsKnRSW2uEGUNi', 'Fee': '10', 'Flags': 0, 'LastLedgerSequence': 24181431, 'Sequence': 24181139, 'SetFlag': 8, 'SigningPubKey': 'EDD0782CAE3F042E194D94377B3FAD7BC4CE9D850BD4E5C352861FB18BBFEB41EA', 'TransactionType': 'AccountSet', 'TxnSignature': '1A0794614BBCE8A17B3FDCE2E1FCB4C39DF36E2DAACD94D6C86F67E075726A7731C59B5E628A1E10DD151FF27764325F16CCE557E9055C400FAD6BAD200FE70F', 'date': 694724250, 'hash': 'C8BD5BE13069CA8E2A4251B65458E48732AE491FC3A2CA92BB1E60029A0172C2', 'inLedger': 24181413, 'ledger_index': 24181413, 'meta': {'AffectedNodes': [{'ModifiedNode': {'FinalFields': {'Account': 'r9WZ5PdJHyL1LodqQxL4TsKnRSW2uEGUNi', 'Balance': '999999990', 'Flags': 8388608, 'OwnerCount': 0, 'Sequence': 24181140}, 'LedgerEntryType': 'AccountRoot', 'LedgerIndex': 'A09024BCB3862212CFD0D3FD3EDE1C4300836AE0CF4E86D81FA9E21C413FC7FA', 'PreviousFields': {'Balance': '1000000000', 'Flags': 0, 'Sequence': 24181139}

In [26]:
#Receiver extends trustline to issuer to accept new token
from xrpl.models.transactions import TrustSet
trustset = TrustSet.from_dict({
    "account": receiver.classic_address,
    "limit_amount": {
        "issuer": issuer.classic_address,
        "currency": token_symbol,
        "value": amount
    }
})
signed_trustset = await safe_sign_and_autofill_transaction(trustset, receiver, client)
response = await send_reliable_submission(signed_trustset, client)
print(response)

Response(status=<ResponseStatus.SUCCESS: 'success'>, result={'Account': 'rwHtUmDVoFjP2jBCQcuWuRDbVU26svAGk9', 'Fee': '10', 'Flags': 0, 'LastLedgerSequence': 24181643, 'LimitAmount': {'currency': '5A494E4300000000000000000000000000000000', 'issuer': 'r9WZ5PdJHyL1LodqQxL4TsKnRSW2uEGUNi', 'value': '13000'}, 'Sequence': 24181141, 'SigningPubKey': 'ED88C40AFB411C8A51F1F57A555D83D27A57FCDB180E88FD2657D2CC51FC7DBF5B', 'TransactionType': 'TrustSet', 'TxnSignature': 'B23AB01D3A98FE594E6A799C8D3189AE4BA256643046E97B6156508A5635E19BBB4C4F507F41AF8E14606078A111E3F268B14076680340FA29487EE61D4CE608', 'date': 694724912, 'hash': '3A4EA68FDF68EE84F4DCDD15A9C6FA61412E5004875A22DA71BFE21FCEFC99BD', 'inLedger': 24181625, 'ledger_index': 24181625, 'meta': {'AffectedNodes': [{'CreatedNode': {'LedgerEntryType': 'DirectoryNode', 'LedgerIndex': '709EADE440953222F7E9BC2B9C7DF15B6974A033FC1011EAF93CE5D6D25A7143', 'NewFields': {'Owner': 'rwHtUmDVoFjP2jBCQcuWuRDbVU26svAGk9', 'RootIndex': '709EADE440953222F7E9BC2B9

C:\Users\Kris\AppData\Local\Temp/ipykernel_10776/1011151164.py:11: RuntimeWarning: coroutine 'send_reliable_submission' was never awaited
  response = await send_reliable_submission(signed_trustset, client)
C:\Users\Kris\AppData\Local\Temp/ipykernel_10776/1011151164.py:11: RuntimeWarning: coroutine 'safe_sign_and_autofill_transaction' was never awaited
  response = await send_reliable_submission(signed_trustset, client)


In [28]:
#Check receiver account trust lines
from xrpl.models.requests import AccountLines
from pprint import pprint
al = AccountLines(account=receiver.classic_address)
response = await client.request(al)
pprint(response.result)

{'account': 'rwHtUmDVoFjP2jBCQcuWuRDbVU26svAGk9',
 'ledger_current_index': 24181769,
 'lines': [{'account': 'r9WZ5PdJHyL1LodqQxL4TsKnRSW2uEGUNi',
            'balance': '0',
            'currency': '5A494E4300000000000000000000000000000000',
            'limit': '13000',
            'limit_peer': '0',
            'no_ripple': False,
            'no_ripple_peer': False,
            'quality_in': 0,
            'quality_out': 0}],
 'validated': False}


In [40]:
#Issue payment of zinc to receiver and sign tx
from xrpl.models.transactions import Payment
from xrpl.models.amounts import IssuedCurrencyAmount
zinc_amount = IssuedCurrencyAmount(value = "390", issuer= issuer.classic_address, currency= token_symbol)
payment = Payment(account= issuer.classic_address, destination= receiver.classic_address, amount= zinc_amount)
signed_payment = await safe_sign_and_autofill_transaction(payment, issuer, client)
response = await send_reliable_submission(signed_payment, client)
pprint(response)

Response(status=<ResponseStatus.SUCCESS: 'success'>,
         result={'Account': 'r9WZ5PdJHyL1LodqQxL4TsKnRSW2uEGUNi',
                 'Amount': {'currency': '5A494E4300000000000000000000000000000000',
                            'issuer': 'r9WZ5PdJHyL1LodqQxL4TsKnRSW2uEGUNi',
                            'value': '390'},
                 'Destination': 'rwHtUmDVoFjP2jBCQcuWuRDbVU26svAGk9',
                 'Fee': '10',
                 'Flags': 0,
                 'LastLedgerSequence': 24182518,
                 'Sequence': 24181143,
                 'SigningPubKey': 'EDD0782CAE3F042E194D94377B3FAD7BC4CE9D850BD4E5C352861FB18BBFEB41EA',
                 'TransactionType': 'Payment',
                 'TxnSignature': '8F919474505F2EF1123251A8BC88C768CBD8617E958CFB8A9ED796BA782AD0ACF40FB91D6F10D58BE5C74C876993052EEC6D600A666285E7C2D1B98528F6E602',
                 'date': 694727631,
                 'hash': '3545F40C97972EC0BF4787EE83AB81AC5818B727A3AACB9D9C524247B430A185',
              

In [42]:
# Generating another wallet and setting a trustline
account3 = await generate_faucet_wallet(client)
print(account3)

public_key: ED074EE9AA0850AE23857FAB844273FFBFFE2FA4FC080448454739E9ABE1F7E156
private_key: -HIDDEN-
classic_address: rHSS9YqaAWs69g2WPpJawTte75sQc6AWXS


In [43]:
#Account3 sets trustline to issuer to accept $zinc
from xrpl.models.transactions import TrustSet
trustset = TrustSet.from_dict({
    "account": account3.classic_address,
    "limit_amount": {
        "issuer": issuer.classic_address,
        "currency": token_symbol,
        "value": amount
    }
})
signed_trustset = await safe_sign_and_autofill_transaction(trustset, account3, client)
response = await send_reliable_submission(signed_trustset, client)
print(response)

Response(status=<ResponseStatus.SUCCESS: 'success'>, result={'Account': 'rHSS9YqaAWs69g2WPpJawTte75sQc6AWXS', 'Fee': '10', 'Flags': 0, 'LastLedgerSequence': 24183100, 'LimitAmount': {'currency': '5A494E4300000000000000000000000000000000', 'issuer': 'r9WZ5PdJHyL1LodqQxL4TsKnRSW2uEGUNi', 'value': '13000'}, 'Sequence': 24183051, 'SigningPubKey': 'ED074EE9AA0850AE23857FAB844273FFBFFE2FA4FC080448454739E9ABE1F7E156', 'TransactionType': 'TrustSet', 'TxnSignature': '29ADE4F1A166FB5BD4188FA7F6993847668282D67FF71D723CCE98F4A6BFB4294BDA106E49485CD916F5EE9ABC1CC5085C9BB86912536E649D18E4D4EC18540B', 'date': 694729450, 'hash': '9BC78808B4597082FB26A58ABFDC1F6D99888BBD58323253C965876008DA9CBD', 'inLedger': 24183082, 'ledger_index': 24183082, 'meta': {'AffectedNodes': [{'CreatedNode': {'LedgerEntryType': 'RippleState', 'LedgerIndex': '0B086FA437E82A9C696C9A23CC01EA5D5F6FA258F3D801165A63B653732E3C42', 'NewFields': {'Balance': {'currency': '5A494E4300000000000000000000000000000000', 'issuer': 'rrrrrrrrr

In [44]:
#Transfer $zinc to account3
zinc_transfer = IssuedCurrencyAmount(value= "20", issuer= issuer.classic_address, currency=token_symbol)
payment2 = Payment(account= receiver.classic_address, destination= account3.classic_address, amount= zinc_transfer)
signed_payment2= await safe_sign_and_autofill_transaction(payment2, receiver, client)
response = await send_reliable_submission(signed_payment2, client)
pprint(response)

Response(status=<ResponseStatus.SUCCESS: 'success'>,
         result={'Account': 'rwHtUmDVoFjP2jBCQcuWuRDbVU26svAGk9',
                 'Amount': {'currency': '5A494E4300000000000000000000000000000000',
                            'issuer': 'r9WZ5PdJHyL1LodqQxL4TsKnRSW2uEGUNi',
                            'value': '20'},
                 'Destination': 'rHSS9YqaAWs69g2WPpJawTte75sQc6AWXS',
                 'Fee': '10',
                 'Flags': 0,
                 'LastLedgerSequence': 24183296,
                 'Sequence': 24181142,
                 'SigningPubKey': 'ED88C40AFB411C8A51F1F57A555D83D27A57FCDB180E88FD2657D2CC51FC7DBF5B',
                 'TransactionType': 'Payment',
                 'TxnSignature': 'FA0AA4BC33490CDB8BBF416D04C703ACC8E937B19B1BE23E0CD65FFF2A23BAEA793ED19BD892C6E1668DF62B96B40EA5CC00D870DC4E2AFD8F610434699F9901',
                 'date': 694730061,
                 'hash': 'A0F169174776DFE70BDEE867923050FF17C07941111FD2594BC2FF2EB8A10A68',
               